In [1]:
import pandas as pd   
import numpy as np

from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import PorterStemmer

def clean_review(raw_review):
	example1 = BeautifulSoup(raw_review, 'html.parser').get_text()
	# print type(example1)
	# Use regular expressions to do a find-and-replace
	letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
	                      " ",                   # The pattern to replace it with
	                      example1 )  # The text to search
	# print letters_only
	lower_case = letters_only.lower()        # Convert to lower case
	words = lower_case.split()               # Split into words
	# print words
	# print stopwords.words("english")
	words = [w for w in words if not w in stopwords.words("english")]
	# print words
	from nltk.stem.porter import PorterStemmer
	porter_stemmer = PorterStemmer()
	stemmed_words = []
	for i in range(len(words)):
		stemmed_words.append(porter_stemmer.stem(words[i]))
	return( " ".join( stemmed_words ))


train = pd.read_csv("data/IMDB/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
# print train.columns.values
num_reviews = train["review"].size
clean_reviews = []
for i in range(num_reviews):
	if( (i+1)%1000 == 0 ):
		print("Review %d of %d\n" % ( i+1, num_reviews )   )                                                       
	clean_reviews.append(clean_review(train["review"][i]))

(25000, 3)


Review 1000 of 25000



Review 2000 of 25000



In [25]:
print(len(clean_reviews))
print(clean_reviews[8597][6])
for i in range(8597):
    clean_reviews[i] = clean_reviews[i].split(' ')
print(clean_reviews[1])

8598
i
['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']


In [26]:
from lib.tensorflowglove import tf_glove
model = tf_glove.GloVeModel(embedding_size=300, context_size=10)
model.fit_to_corpus(clean_reviews)

In [27]:
model.train(num_epochs=100)

KeyboardInterrupt: 

In [17]:
import matplotlib
%matplotlib inline
model.generate_tsne()

In [18]:
model.embeddings

array([[-0.58207422,  1.08487749, -0.68133765, ..., -1.2772572 ,
         0.05729133, -0.44729194],
       [ 0.15404087, -0.85265279, -0.85540909, ..., -0.12607348,
         0.78001708,  0.1322436 ],
       [-0.39311522, -0.42893636,  0.52570939, ..., -0.39255959,
         1.53533649,  1.82403588],
       ..., 
       [ 0.28314537,  0.41172779,  0.28595892, ..., -0.11498308,
        -0.75357479,  1.15168238],
       [-0.12832922,  1.68767095,  0.54099941, ...,  1.06163406,
         0.64280045,  0.27908194],
       [ 0.91403013, -0.4698227 , -0.0113878 , ...,  0.11128509,
         0.94589418,  0.1831885 ]], dtype=float32)

In [22]:
model.embeddings[model.id_for_word('k')]

array([-0.03222942, -0.36123627, -0.08965302, -0.87305146,  0.10418586,
       -0.82008815, -0.26724911,  1.11199832, -0.82340115, -0.44559732,
       -0.30026934,  1.0071888 , -0.13929728, -0.62022358, -0.85671866,
        0.78747004, -0.90620524,  0.7232579 , -0.13922298,  1.06302559,
       -0.40877268,  1.02106941, -1.13613915,  0.50750464, -0.81685674,
        0.03576756, -0.19330762, -0.2572692 ,  0.02214804,  0.06502384,
       -0.70460033,  0.52478373,  0.76222306,  0.31561702,  1.06144416,
        0.78358477,  0.20295665,  1.22286308,  0.75359458, -1.78022742,
       -0.55827641, -0.22303838, -0.15053667, -1.64959645,  0.81298316,
        0.4726342 , -0.06501664, -0.35705787,  0.66635913, -0.15118098,
       -1.36974335, -0.3380996 ,  1.25974858,  0.45577219,  1.04450655,
       -0.25281462,  1.54658175, -0.00569487,  0.40391719,  0.82098067,
       -0.73775345, -0.66108769, -0.67710793, -1.08653831,  1.14112234,
       -0.94671381, -0.71269953, -0.8652851 , -0.42442209,  1.14

In [24]:
print(clean_reviews[1].split(' '))

['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']
